In [1]:
from datasets import load_dataset
import pandas as pd
import logging
import time
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import LLMChain
from langchain_groq import ChatGroq
import os
from langchain_cohere import ChatCohere

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from IPython.display import display, HTML
import json

d:\Teias\Thesis\self-explaination-thesis\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
file_path = '../../datasets/question answering/belebele/pes_arab_dataset.csv'  # Path for the CSV file
data = pd.read_csv(file_path)

# Sample 50 entries from the dataset
sampled_data = data.sample(n=50, random_state=42)

sampled_data

,link,question_number,flores_passage,question,mc_answer1,mc_answer2,mc_answer3,mc_answer4,correct_answer_num,dialect,ds
70,https://en.wikibooks.org/wiki/High_School_Eart...,1,بشر هزاران هزار سال از عدسی‌ها برای بزرگ‌نمایی...,کدام یک از موارد زیر را انسان‌ها در قرن شانزده...,استفاده از عدسی‌ها,بزرگنمایی اشیاء,ساخت عدسی‌ها,ساخت تلسکوپ,4,pes_Arab,2023-06-07
827,https://en.wikivoyage.org/wiki/Paraguay,1,گوارانی‌ها مهم‌ترین گروه بومی ساکن پاراگوئه شر...,دوران استعمارگری که سه قرن طول کشید توسط کدام ...,پاراگوئه,Guaycurú,گورانی‌ها,فاتحان اسپانیایی,4,pes_Arab,2023-06-07
231,https://en.wikibooks.org/wiki/Wikijunior:Solar...,2,خورشید پوسته‌ای شبیه کره زمین ندارد که بتوانید...,با توجه به متن، کدام قسمت از خورشید از زمین قا...,پوسته,پلاسما,فوتوسفر,مرکز,3,pes_Arab,2023-06-07
588,https://en.wikinews.org/wiki/Seven_killed_in_C...,2,آن مرد ظاهراً یک خودروی سه‌چرخ مسلح به مواد من...,مقامات چه اطلاعاتی درخصوص حمله تأیید کرده بودند؟,هویت هم‌دست‌ها,قومیت مظنون,انگیزه حمله,نام و نام خانوادگی مظنون,2,pes_Arab,2023-06-07
39,https://en.wikibooks.org/wiki/Environmental_th...,1,ما می توانیم زندگی دوستانه تری را با محیط زیست...,با توجه به متن، کدام مورد راه‌حلی قابل اجرا بر...,تحریم انتخاب‌های سبک زندگی مصرف‌گرایانه,کنشگری,یافتن ریشه مسئله,تبدیل شدن به متحد جنبش محیط زیست,3,pes_Arab,2023-06-07
731,https://en.wikivoyage.org/wiki/Downhill_snowsp...,2,ورزش‌های زمستانی روی تپه، از جمله اسکی و اسنوب...,مطابق با متن، حدود چه زمانی اسکی‌بازی اولین با...,قرن هجدهم,5000 سال قبل از میلاد,1861,قرن هفدهم,4,pes_Arab,2023-06-07
299,https://en.wikinews.org/wiki/Australian_Big_Br...,2,ACMA همچنین نتیجه گرفت که علیرغم اینکه این وید...,برای اینکه رسانه تحت مقوله محتوای اینترنتی لحا...,باید از طریق اینترنت پخش شود,باید به صورت فیزیکی در سروری قرار گیرد,نمی‌تواند روی وبسایت ذخیره شود,باید توسط ACMA تأیید شود,2,pes_Arab,2023-06-07
110,https://en.wikibooks.org/wiki/Mysticism/What_i...,2,عرفان عبارت است از جستجوی ارتباط، شناخت یا درک...,با توجه به متن، کدام یک از موارد زیر مورد توجه...,جستجوی شهود به واقعیت الهی,طلب بینش از طریق تجربیات مشترک و همگانی,تأکید بر حالت سعادتمند خودآگاه,طلب درک آگاهانه الوهیت,2,pes_Arab,2023-06-07
72,https://en.wikibooks.org/wiki/History_of_Islam...,1,محمد عمیقا به امور فراتر از این زندگی دنیوی عل...,بر اساس متن، غار محمد بیانگر چه چیزی درباره اع...,او برای آرامش و انزوا ارزش قائل بود,صدای بیرون به تأمل او کمک می‌کرد,به زمان خیلی کمی برای تنهایی نیاز داشت,نگران این بود که دیده یا شنیده شود,1,pes_Arab,2023-06-07
86,https://en.wikibooks.org/wiki/Introduction_to_...,1,کودکان از زمان کودکی نسبت به نژاد و کلیشه های ...,با توجه به نظریه ارائه شده در متن، برای کودکان...,کودک سعی می‌کند از درگیر شدن در رفتارهای تهدید...,احتمال بیشتری وجود دارد که کودک درگیر رفتارهای...,کودک در موقعیت‌های اجتماعی کناره‌گیری می‌کند,کودک از تعامل با سایر اقلیت‌های نژادی امتناع خ...,2,pes_Arab,2023-06-07


In [2]:
os.environ['GROQ_API_KEY'] = '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'

In [3]:
from groq import Groq

client = Groq(
    api_key=os.environ.get("GROQ_API_KEY"),
)

In [ ]:
# Function to predict the answer and save the full generation to a file
def predict_answer_and_save(passage, question, options, client, model_name):
    logging.info(f"Starting prediction for passage: {passage[:50]}... and question: {question}")
    try:
        # Prepare the messages for the model
        res = client.chat.completions.create(
            model=model_name,  # Replace with the correct model
            messages=[
                {
                    "role": "system",
                    "content": (
                        "You will receive a `QUESTION` with multiple `ANSWER CHOICES` and a supporting `CONTEXT`. "
                        "Your task is to determine the correct answer based on the information in the context and to explain "
                        "your reasoning step-by-step, referencing key details from the context.\n\n"
                        "Follow these instructions for your response:\n\n"
                        "Select only one answer choice (1, 2, 3, or 4) based on your understanding of the context.\n"
                        "Provide a detailed, step-by-step explanation of how you arrived at the answer, focusing on the logical reasoning process. "
                        "Include references to relevant parts of the context to support each reasoning step. `The explanation must be in Persian.`\n"
                        "Identify and quote the most relevant parts of the context that were essential in answering the question. "
                        "Only include the specific information that directly supports your chosen answer. `Ensure this is in Persian.`\n"
                        "List the most important keywords from the context that are essential for understanding and answering the question. "
                        "---\n\n"
                        "`Guidelines for Explanation`:\n"
                        "- Make sure your explanation is complete, covering all reasoning steps used to reach the answer.\n"
                        "- Ensure your reasoning is relevant and directly tied to the context, avoiding any unrelated details.\n"
                        "- Keep the explanation clear and logical, with each step following naturally from the previous one.\n"
                        "- Give json output format"
                    )
                },
                {
                    "role": "user",
                    "content": (
                        f"CONTEXT:\n{passage}\n\n"
                        f"QUESTION:\n{question}\n\n"
                        f"ANSWER CHOICES:\n1-{options[0]}\n2-{options[1]}\n3-{options[2]}\n4-{options[3]}"
                    )
                }
            ],
            response_format={
                "type": "json_object",
                "schema": {
                    "type": "object",
                    "properties": {
                        "answer": {"type": "integer"},
                        "explanation": {"type": "string"},
                        "highlight": {"type": "string"},
                        "keywords": {"type": "array", "items": {"type": "string"}}
                    },
                    "required": ["answer", "explanation", "highlight", "keywords"]
                },
            },
        )
        
        # Assuming the response is formatted as expected
        #response = res.message.content[0].text.strip()
        response = res.choices[0].message.content
        
        # Output the model's response for preview
        print("Response from model:", response)
        
        # Parse the response into a JSON-like object (if it's in the correct format)
        response_json = json.loads(response)
        
        # Example: Print the response as a dictionary
        print("Parsed response:", response_json)
        return response_json
        
    except Exception as e:
        logging.error(f"Error during prediction: {e}")
        return None


In [68]:
# Sample data to test the function
passage = "The sun provides light and energy to the plants, which is essential for photosynthesis."
question = "What is the primary role of the sun for plants?"
options = ["A: Provides light", "B: Provides water", "C: Provides nutrients", "D: Provides oxygen"]

# Run the prediction function
result = predict_answer_and_save(passage, question, options, client, model_name='llama3-8b-8192')

# Check the result
if result:
    print("Predicted Answer:", result["answer"])
    print("Explanation:", result["explanation"])
    print("Highlight:", result["highlight"])
    print("Keywords:", result["keywords"])
else:
    print("Prediction failed.")

Response from model: {
   "Answer": 1,
   "Explanation": "بر اساس متن، خورشید نقش اصلی خود را در رشد گیاهان ایفا می‌کند. دومین گزینه، از متن جدا شده و در متن هیچ اشاره‌ای بهrole خورشید در تامین آب گیاهان شده است. سومین گزینه نیز، در متن هیچ اشاره‌ای به تامین مواد مغذی توسط خورشید نداشته‌است. четвертый گزینه نیز، خورشید را به عنوان منبع اکسیژن مشخص می‌کند، در حالی که خورشید در تولید اکسیژن دخیل نیست.，因此هیمه role اصلی خورشید برای گیاه‌ها تامین نور است.",
   "Key Passages":
   "The sun provides light and energy to the plants, which is essential for photosynthesis.",
   "Important Keywords": ["خورشید","گیاهان","نور","-energy","photosynthesis"]
}
Parsed response: {'Answer': 1, 'Explanation': 'بر اساس متن، خورشید نقش اصلی خود را در رشد گیاهان ایفا می\u200cکند. دومین گزینه، از متن جدا شده و در متن هیچ اشاره\u200cای بهrole خورشید در تامین آب گیاهان شده است. سومین گزینه نیز، در متن هیچ اشاره\u200cای به تامین مواد مغذی توسط خورشید نداشته\u200cاست. четвертый گزینه نیز، خورشید را به عنوان منبع اکس

KeyError: 'answer'

In [4]:
df = pd.read_csv('sample_result.csv')

In [5]:
import logging
import json

# Function to predict only the final answer
def predict_answer_by_own_knowladge(passage, question, options, client, model_name):
    logging.info(f"Starting prediction for passage: {passage[:50]}... and question: {question}")
    try:
        # Prepare the messages for the model
        res = client.chat.completions.create(
            model=model_name,
            messages=[
                {
                    "role": "system",
                    "content": (
                        "You will receive a `QUESTION` with multiple `ANSWER CHOICES` and a supporting `CONTEXT`. "
                        "Your task is to determine the correct answer based on the information in the context. "
                        "Select only one answer choice (1, 2, 3, or 4) based on your understanding of the context. "
                        "Return only the answer as an integer in JSON format."
                    )
                },
                {
                    "role": "user",
                    "content": (
                        f"CONTEXT:\n{passage}\n\n"
                        f"QUESTION:\n{question}\n\n"
                        f"ANSWER CHOICES:\n1-{options[0]}\n2-{options[1]}\n3-{options[2]}\n4-{options[3]}"
                    )
                }
            ],
            response_format={
                "type": "json_object",
                "schema": {
                    "type": "object",
                    "properties": {
                        "answer": {"type": "integer"}
                    },
                    "required": ["answer"]
                },
            },
        )
        
        # Extract the response content
        response = res.choices[0].message.content
        
        # Output the model's response for preview
        print("Response from model (Own):", response)
        
        # Parse the response into a JSON object
        response_json = json.loads(response)
        
        # Extract and return just the answer
        answer = response_json["answer"]
        return answer
        
    except Exception as e:
        logging.error(f"Error during prediction: {e}")
        return None

In [6]:
import logging
import json

# Function to predict only the final answer
def predict_answer_by_highlight_of_context(passage, question, options, client, model_name):
    logging.info(f"Starting prediction for passage: {passage[:50]}... and question: {question}")
    try:
        # Prepare the messages for the model
        res = client.chat.completions.create(
            model=model_name,
            messages=[
                {
                    "role": "system",
                    "content": (
                        "You will receive a `QUESTION` with multiple `ANSWER CHOICES` and a supporting `Highlight` of context. "
                        "Your task is to determine the correct answer based on the information in the Highlight of context. "
                        "Select only one answer choice (1, 2, 3, or 4) based on your understanding of the context. "
                        "Return only the answer as an integer in JSON format."
                    )
                },
                {
                    "role": "user",
                    "content": (
                        f"HIGHLIGHT OF CONTEXT:\n{passage}\n\n"
                        f"QUESTION:\n{question}\n\n"
                        f"ANSWER CHOICES:\n1-{options[0]}\n2-{options[1]}\n3-{options[2]}\n4-{options[3]}"
                    )
                }
            ],
            response_format={
                "type": "json_object",
                "schema": {
                    "type": "object",
                    "properties": {
                        "answer": {"type": "integer"}
                    },
                    "required": ["answer"]
                },
            },
        )
        
        # Extract the response content
        response = res.choices[0].message.content
        
        # Output the model's response for preview
        print("Response from model (Highlight):", response)
        
        # Parse the response into a JSON object
        response_json = json.loads(response)
        
        # Extract and return just the answer
        answer = response_json["answer"]
        return answer
        
    except Exception as e:
        logging.error(f"Error during prediction: {e}")
        return None

In [7]:
def make_request_with_retry(func, *args, retries=3, delay=10):
    for attempt in range(retries):
        try:
            return func(*args)
        except ConnectionError as e:
            logging.warning(f"Attempt {attempt + 1} failed: {e}. Retrying in {delay} seconds...")
            time.sleep(delay)
    logging.error("Max retries exceeded")
    raise ConnectionError("Max retries exceeded")

In [8]:
df.head()

,Passage,Question,Original Answer,Predicted Answer,Option 1,Option 2,Option 3,Option 4,Explanation:,Highlight:,Keywords:
0,بشر هزاران هزار سال از عدسی‌ها برای بزرگ‌نمایی...,کدام یک از موارد زیر را انسان‌ها در قرن شانزده...,4,4,استفاده از عدسی‌ها,بزرگنمایی اشیاء,ساخت عدسی‌ها,ساخت تلسکوپ,در متن به صراحت اشاره شده است که ساخت تلسکوپ ه...,با این حال، اولین تلسکوپ های واقعی در اواخر قر...,"['عدسی', 'بزرگنمایی', 'تلسکوپ', 'قرن شانزدهم',..."
1,گوارانی‌ها مهم‌ترین گروه بومی ساکن پاراگوئه شر...,دوران استعمارگری که سه قرن طول کشید توسط کدام ...,4,4,پاراگوئه,Guaycurú,گورانی‌ها,فاتحان اسپانیایی,در متن به صراحت ذکر شده است که دوره استعماری س...,اسپانیایی‌ها دوره استعماری را شروع کردند که سه...,"['فاتحان اسپانیایی', 'دوره استعماری', 'قرن 16'..."
2,خورشید پوسته‌ای شبیه کره زمین ندارد که بتوانید...,با توجه به متن، کدام قسمت از خورشید از زمین قا...,3,3,پوسته,پلاسما,فوتوسفر,مرکز,در متن به وضوح ذکر شده است که «وقتی به خورشید ...,وقتی به خورشید نگاه می‌کنیم قسمت بیرونی که می‌...,"['فوتوسفر', 'قسمت بیرونی', 'قابل مشاهده']"
3,آن مرد ظاهراً یک خودروی سه‌چرخ مسلح به مواد من...,مقامات چه اطلاعاتی درخصوص حمله تأیید کرده بودند؟,2,2,هویت هم‌دست‌ها,قومیت مظنون,انگیزه حمله,نام و نام خانوادگی مظنون,در متن ارائه شده، به دو مورد از اطلاعات تایید ...,مرد مظنون به انفجار بمب، پس از تحمل جراحات ناش...,"['مرد مظنون', 'بازداشت', 'گروه قومی اویغور']"
4,ما می توانیم زندگی دوستانه تری را با محیط زیست...,با توجه به متن، کدام مورد راه‌حلی قابل اجرا بر...,3,3,تحریم انتخاب‌های سبک زندگی مصرف‌گرایانه,کنشگری,یافتن ریشه مسئله,تبدیل شدن به متحد جنبش محیط زیست,در این متن، به دنبال راه‌حلی برای مسائل زیست‌م...,با این وجود، اگر تنها به دنبال یک راه‌حل موقت ...,"['راه\u200cحل موقت', 'ریشه مشکلات', 'فعالیت\u2..."


In [9]:
def apply_prediction_and_save(df, mode="just_predict", result_name = 'check_utility'):
    results = []

    for index, row in df.iterrows():
        print("GDFGDFGDFGDFGDFGDFG")
        passage = row['Passage']
        question = row['Question']
        options = [row['Option 1'], row['Option 2'], row['Option 3'], row['Option 4']]
        original_answer = row['Original Answer']
        highlight_context = row['Highlight:']

        logging.info(f"Processing row {index+1}/{len(df)}: {passage[:50]}...")

        # Select the appropriate method
        if mode == "just_predict":
            result1 = make_request_with_retry(predict_answer_by_own_knowladge, passage, question, options, client, "llama3-8b-8192")
            time.sleep(7)
            result2 = make_request_with_retry(predict_answer_by_highlight_of_context, highlight_context, question, options, client, "llama3-8b-8192")
            time.sleep(10)
        elif mode == "predict_and_explain":
            #result = make_request_with_retry(predict_answer_by_highlight_of_context, highlight_context, question, options, "llama3-8b-8192")
            time.sleep(10)
        else:
            logging.error(f"Invalid mode: {mode}")
            predicted_answer = "error"

        results.append({
            "Passage": passage,
            "Question": question,
            "Original Answer": original_answer,
            "Predicted Answer": row["Predicted Answer"],
            "Predicted Answer By Highlight" : result2,
            "Predicted Answer By Own" : result1,
            "Option 1": options[0],
            "Option 2": options[1],
            "Option 3": options[2],
            "Option 4": options[3],
            "Explanation": row["Explanation:"],
            "Highlight": row["Highlight:"],
            "Keywords":row["Keywords:"]
        })

        logging.info(f"Row {index+1} processed successfully.")

    # Save final results as DataFrame
    result_df = pd.DataFrame(results)
    output_file_name = result_name + ".csv"
    result_df.to_csv(output_file_name, index=False)
    logging.info("Results saved to {o}".format(o = output_file_name))

    return result_df

In [10]:
apply_prediction_and_save(df)

GDFGDFGDFGDFGDFGDFG
Response from model (Own): {
   "answer": 4
}


ERROR:root:Error during prediction: Error code: 400 - {'error': {'message': "Failed to generate JSON. Please adjust your prompt. See 'failed_generation' for more details.", 'type': 'invalid_request_error', 'code': 'json_validate_failed', 'failed_generation': '{\n   "1"\n}'}}


GDFGDFGDFGDFGDFGDFG
Response from model (Own): {
   "answer": 4
}
Response from model (Highlight): {
   "answer": 4
}
GDFGDFGDFGDFGDFGDFG
Response from model (Own): {
   "answer": 3
}
Response from model (Highlight): {
   "answer": 3
}
GDFGDFGDFGDFGDFGDFG
Response from model (Own): {
   "answer": 2
}
Response from model (Highlight): {
   "answer": 2
}
GDFGDFGDFGDFGDFGDFG
Response from model (Own): {
   "answer": 3
}
Response from model (Highlight): {
   "answer": 3
}
GDFGDFGDFGDFGDFGDFG
Response from model (Own): {
   "answer": 2
}
Response from model (Highlight): {
   "answer": 4
}
GDFGDFGDFGDFGDFGDFG
Response from model (Own): {
   "answer": 2
}
Response from model (Highlight): {
   "answer": 2
}
GDFGDFGDFGDFGDFGDFG
Response from model (Own): {
   "answer": 2
}
Response from model (Highlight): {
   "answer": 2
}
GDFGDFGDFGDFGDFGDFG
Response from model (Own): {
   "answer": 1
}
Response from model (Highlight): {
   "answer": 1
}
GDFGDFGDFGDFGDFGDFG
Response from model (Own): {
   "ans

ERROR:root:Error during prediction: Error code: 400 - {'error': {'message': "Failed to generate JSON. Please adjust your prompt. See 'failed_generation' for more details.", 'type': 'invalid_request_error', 'code': 'json_validate_failed', 'failed_generation': '{\n   "1"\n}'}}


GDFGDFGDFGDFGDFGDFG
Response from model (Own): {
   "answer": 3
}
Response from model (Highlight): {
   "answer": 3
}
GDFGDFGDFGDFGDFGDFG
Response from model (Own): {
   "answer": 1
}
Response from model (Highlight): {
   "answer": 1
}
GDFGDFGDFGDFGDFGDFG
Response from model (Own): {
   "answer": 1
}
Response from model (Highlight): {
   "answer": 4
}
GDFGDFGDFGDFGDFGDFG
Response from model (Own): {
   "answer": 3
}
Response from model (Highlight): {
   "answer": 4
}
GDFGDFGDFGDFGDFGDFG
Response from model (Own): {
   "answer": 4
}
Response from model (Highlight): {
   "answer": 4
}
GDFGDFGDFGDFGDFGDFG
Response from model (Own): {
   "answer": 2
}
Response from model (Highlight): {
   "answer": 2
}
GDFGDFGDFGDFGDFGDFG
Response from model (Own): {
   "answer": 3
}
Response from model (Highlight): {
   "answer": 3
}
GDFGDFGDFGDFGDFGDFG
Response from model (Own): {
   "answer": 3
}
Response from model (Highlight): {
   "answer": 3
}
GDFGDFGDFGDFGDFGDFG
Response from model (Own): {
   "ans

ERROR:root:Error during prediction: Error code: 400 - {'error': {'message': "Failed to generate JSON. Please adjust your prompt. See 'failed_generation' for more details.", 'type': 'invalid_request_error', 'code': 'json_validate_failed', 'failed_generation': '{\n   "1"\n}'}}


Response from model (Highlight): {
   "answer": 4
}
GDFGDFGDFGDFGDFGDFG
Response from model (Own): {
   "answer": 1
}
Response from model (Highlight): {
   "answer": 1
}
GDFGDFGDFGDFGDFGDFG
Response from model (Own): {
   "answer": 3
}
Response from model (Highlight): {
   "answer": 3
}
GDFGDFGDFGDFGDFGDFG
Response from model (Own): {
   "answer": 1
}
Response from model (Highlight): {
   "answer": 1
}


,Passage,Question,Original Answer,Predicted Answer,Predicted Answer By Highlight,Predicted Answer By Own,Option 1,Option 2,Option 3,Option 4,Explanation,Highlight,Keywords
0,بشر هزاران هزار سال از عدسی‌ها برای بزرگ‌نمایی...,کدام یک از موارد زیر را انسان‌ها در قرن شانزده...,4,4,NaN,4.0,استفاده از عدسی‌ها,بزرگنمایی اشیاء,ساخت عدسی‌ها,ساخت تلسکوپ,در متن به صراحت اشاره شده است که ساخت تلسکوپ ه...,با این حال، اولین تلسکوپ های واقعی در اواخر قر...,"['عدسی', 'بزرگنمایی', 'تلسکوپ', 'قرن شانزدهم',..."
1,گوارانی‌ها مهم‌ترین گروه بومی ساکن پاراگوئه شر...,دوران استعمارگری که سه قرن طول کشید توسط کدام ...,4,4,4.0,4.0,پاراگوئه,Guaycurú,گورانی‌ها,فاتحان اسپانیایی,در متن به صراحت ذکر شده است که دوره استعماری س...,اسپانیایی‌ها دوره استعماری را شروع کردند که سه...,"['فاتحان اسپانیایی', 'دوره استعماری', 'قرن 16'..."
2,خورشید پوسته‌ای شبیه کره زمین ندارد که بتوانید...,با توجه به متن، کدام قسمت از خورشید از زمین قا...,3,3,3.0,3.0,پوسته,پلاسما,فوتوسفر,مرکز,در متن به وضوح ذکر شده است که «وقتی به خورشید ...,وقتی به خورشید نگاه می‌کنیم قسمت بیرونی که می‌...,"['فوتوسفر', 'قسمت بیرونی', 'قابل مشاهده']"
3,آن مرد ظاهراً یک خودروی سه‌چرخ مسلح به مواد من...,مقامات چه اطلاعاتی درخصوص حمله تأیید کرده بودند؟,2,2,2.0,2.0,هویت هم‌دست‌ها,قومیت مظنون,انگیزه حمله,نام و نام خانوادگی مظنون,در متن ارائه شده، به دو مورد از اطلاعات تایید ...,مرد مظنون به انفجار بمب، پس از تحمل جراحات ناش...,"['مرد مظنون', 'بازداشت', 'گروه قومی اویغور']"
4,ما می توانیم زندگی دوستانه تری را با محیط زیست...,با توجه به متن، کدام مورد راه‌حلی قابل اجرا بر...,3,3,3.0,3.0,تحریم انتخاب‌های سبک زندگی مصرف‌گرایانه,کنشگری,یافتن ریشه مسئله,تبدیل شدن به متحد جنبش محیط زیست,در این متن، به دنبال راه‌حلی برای مسائل زیست‌م...,با این وجود، اگر تنها به دنبال یک راه‌حل موقت ...,"['راه\u200cحل موقت', 'ریشه مشکلات', 'فعالیت\u2..."
5,ورزش‌های زمستانی روی تپه، از جمله اسکی و اسنوب...,مطابق با متن، حدود چه زمانی اسکی‌بازی اولین با...,4,4,4.0,2.0,قرن هجدهم,5000 سال قبل از میلاد,1861,قرن هفدهم,در متن به صراحت اشاره شده است که ورزش اسکی روی...,ورزش اسکی روی تپه حداقل به قرن 17 ام برمی‌گردد,"['اسکی', 'ورزش', 'قرن 17 ام']"
6,ACMA همچنین نتیجه گرفت که علیرغم اینکه این وید...,برای اینکه رسانه تحت مقوله محتوای اینترنتی لحا...,2,2,2.0,2.0,باید از طریق اینترنت پخش شود,باید به صورت فیزیکی در سروری قرار گیرد,نمی‌تواند روی وبسایت ذخیره شود,باید توسط ACMA تأیید شود,در متن به صراحت بیان شده است که برای اینکه محت...,قانون خدمات پخش مقررات محتوای اینترنتی را مشخص...,"['قانون خدمات پخش', 'محتوای اینترنتی', 'سرور',..."
7,عرفان عبارت است از جستجوی ارتباط، شناخت یا درک...,با توجه به متن، کدام یک از موارد زیر مورد توجه...,2,2,2.0,2.0,جستجوی شهود به واقعیت الهی,طلب بینش از طریق تجربیات مشترک و همگانی,تأکید بر حالت سعادتمند خودآگاه,طلب درک آگاهانه الوهیت,در متن به صراحت بیان شده است که عرفان بر تجربه...,در عرفان بر تجربه مستقیم **شخصی** تأکید دارد.,"['تجربه مستقیم', 'حالت هشیاری', 'تجربه شخصی', ..."
8,محمد عمیقا به امور فراتر از این زندگی دنیوی عل...,بر اساس متن، غار محمد بیانگر چه چیزی درباره اع...,1,1,1.0,1.0,او برای آرامش و انزوا ارزش قائل بود,صدای بیرون به تأمل او کمک می‌کرد,به زمان خیلی کمی برای تنهایی نیاز داشت,نگران این بود که دیده یا شنیده شود,متن به وضوح بیان می‌کند که غار حرا، مکانی منزو...,در یک انزوا کامل خواهید بود,"['غار حرا', 'انزوا', 'آرامش', 'تأمل']"
9,کودکان از زمان کودکی نسبت به نژاد و کلیشه های ...,با توجه به نظریه ارائه شده در متن، برای کودکان...,2,2,2.0,2.0,کودک سعی می‌کند از درگیر شدن در رفتارهای تهدید...,احتمال بیشتری وجود دارد که کودک درگیر رفتارهای...,کودک در موقعیت‌های اجتماعی کناره‌گیری می‌کند,کودک از تعامل با سایر اقلیت‌های نژادی امتناع خ...,در این متن، نظریه‌ای مطرح می‌شود که نشان می‌ده...,کودکان از زمان کودکی نسبت به نژاد و کلیشه های ...,"['کلیشه نژادی', 'تأثیر بر رفتار', 'اقلیت نژادی..."


In [12]:
check_utility_df = pd.read_csv('check_utility.csv')

In [14]:
check_utility_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 13 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Passage                        50 non-null     object 
 1   Question                       50 non-null     object 
 2   Original Answer                50 non-null     int64  
 3   Predicted Answer               50 non-null     int64  
 4   Predicted Answer By Highlight  48 non-null     float64
 5   Predicted Answer By Own        49 non-null     float64
 6   Option 1                       50 non-null     object 
 7   Option 2                       50 non-null     object 
 8   Option 3                       50 non-null     object 
 9   Option 4                       50 non-null     object 
 10  Explanation                    50 non-null     object 
 11  Highlight                      50 non-null     object 
 12  Keywords                       50 non-null     objec

In [15]:
import pandas as pd

# Assuming 'df' is the DataFrame containing your dataset

# Function to calculate accuracy
def calculate_accuracy(df):
    # Accuracy based on Original Answer and Predicted Answer
    accuracy_original_vs_predicted = (df['Original Answer'] == df['Predicted Answer']).mean()
    
    # Accuracy based on Original Answer and Predicted Answer By Highlight
    accuracy_original_vs_predicted_highlight = (df['Original Answer'] == df['Predicted Answer By Highlight']).mean()
    
    # Accuracy based on Original Answer and Predicted Answer By Own
    accuracy_original_vs_predicted_own = (df['Original Answer'] == df['Predicted Answer By Own']).mean()
    
    # Print or return the accuracies
    print(f"Accuracy (Original Answer vs Predicted Answer): {accuracy_original_vs_predicted:.4f}")
    print(f"Accuracy (Original Answer vs Predicted Answer By Highlight): {accuracy_original_vs_predicted_highlight:.4f}")
    print(f"Accuracy (Original Answer vs Predicted Answer By Own): {accuracy_original_vs_predicted_own:.4f}")
    
    return accuracy_original_vs_predicted, accuracy_original_vs_predicted_highlight, accuracy_original_vs_predicted_own

# Example usage
accuracy_original_vs_predicted, accuracy_original_vs_predicted_highlight, accuracy_original_vs_predicted_own = calculate_accuracy(check_utility_df)

Accuracy (Original Answer vs Predicted Answer): 0.9600
Accuracy (Original Answer vs Predicted Answer By Highlight): 0.8200
Accuracy (Original Answer vs Predicted Answer By Own): 0.8000


# With Explaination

In [38]:
import logging
import json

# Function to predict only the final answer
def predict_answer_by_explaination(explaination, question, options, client, model_name):
    logging.info(f"Starting prediction for passage: {explaination[:50]}... and question: {question}")
    try:
        # Prepare the messages for the model
        res = client.chat.completions.create(
            model=model_name,
            messages=[
                {
                    "role": "system",
                    "content": (
                        "You will receive a `QUESTION` with multiple `ANSWER CHOICES` and a `EXPLAINATION` guide for question"
                        "Your task is to determine the correct answer based on the information in Explaination. "
                        "Select only one answer choice (1, 2, 3, or 4) based on Explaination."
                        "Return only the answer as an integer in JSON format."
                    )
                },
                {
                    "role": "user",
                    "content": (
                        f"EXPLAINATION:\n{explaination}\n\n"
                        f"QUESTION:\n{question}\n\n"
                        f"ANSWER CHOICES:\n1-{options[0]}\n2-{options[1]}\n3-{options[2]}\n4-{options[3]}"
                    )
                }
            ],
            response_format={
                "type": "json_object",
                "schema": {
                    "type": "object",
                    "properties": {
                        "answer": {"type": "integer"}
                    },
                    "required": ["answer"]
                },
            },
        )
        
        # Extract the response content
        response = res.choices[0].message.content
        
        # Output the model's response for preview
        print("Response from model (Explaination):", response)
        
        # Parse the response into a JSON object
        response_json = json.loads(response)
        
        # Extract and return just the answer
        answer = response_json["answer"]
        return answer
        
    except Exception as e:
        logging.error(f"Error during prediction: {e}")
        return None

In [41]:
def apply_prediction_and_save_2(df, mode="just_predict", result_name = 'check_utility_2'):
    results = []

    for index, row in df.iterrows():
        print(index)
        passage = row['Passage']
        question = row['Question']
        options = [row['Option 1'], row['Option 2'], row['Option 3'], row['Option 4']]
        original_answer = row['Original Answer']
        explaination = row['Explanation']

        #print(passage)
        logging.info(f"Processing row {index+1}/{len(df)}: {explaination[:50]}...")

        # Select the appropriate method
        if mode == "just_predict":
            result3 = make_request_with_retry(predict_answer_by_explaination, explaination, question, options, client, "llama3-8b-8192")
            time.sleep(10)
        elif mode == "predict_and_explain":
            #result = make_request_with_retry(predict_answer_by_highlight_of_context, highlight_context, question, options, "llama3-8b-8192")
            pass  
        else:
            logging.error(f"Invalid mode: {mode}")
            predicted_answer = "error"

        results.append({
            "Passage": passage,
            "Question": question,
            "Original Answer": original_answer,
            "Predicted Answer": row["Predicted Answer"],
            "Predicted Answer By Highlight" : row['Predicted Answer By Highlight'],
            "Predicted Answer By Own" : row['Predicted Answer By Own'],
            "Predicted Answer By Explaination" : result3,
            "Option 1": options[0],
            "Option 2": options[1],
            "Option 3": options[2],
            "Option 4": options[3],
            "Explanation": row["Explanation"],
            "Highlight": row["Highlight"],
            "Keywords":row["Keywords"]
        })

        logging.info(f"Row {index+1} processed successfully.")

    # Save final results as DataFrame
    result_df = pd.DataFrame(results)
    output_file_name = result_name + ".csv"
    result_df.to_csv(output_file_name, index=False)
    logging.info("Results saved to {o}".format(o = output_file_name))

    return result_df

In [42]:
apply_prediction_and_save_2(check_utility_df)

0
Response from model (Explaination): {
   "answer": 4
}
1
Response from model (Explaination): {
   "answer": 4
}
2
Response from model (Explaination): {
   "answer": 3
}
3
Response from model (Explaination): {
   "answer": 2
}
4
Response from model (Explaination): {
   "answer": 3
}
5
Response from model (Explaination): {
   "answer": 4
}
6
Response from model (Explaination): {
   "answer": 2
}
7
Response from model (Explaination): {
   "answer": 2
}
8
Response from model (Explaination): {
   "answer": 1
}
9
Response from model (Explaination): {
   "answer": 2
}
10
Response from model (Explaination): {
   "answer": 3
}
11
Response from model (Explaination): {
   "answer": 2
}
12
Response from model (Explaination): {
   "answer": 3
}
13


ERROR:root:Error during prediction: 'answer'


Response from model (Explaination): {
   "Answer": 3
}
14
Response from model (Explaination): {
   "answer": 2
}
15
Response from model (Explaination): {
   "answer": 3
}
16
Response from model (Explaination): {
   "answer": 4
}
17
Response from model (Explaination): {
   "answer": 1
}
18
Response from model (Explaination): {
   "answer": 1
}
19
Response from model (Explaination): {
   "answer": 1
}
20
Response from model (Explaination): {
   "answer": 2
}
21
Response from model (Explaination): {
   "answer": 1
}
22
Response from model (Explaination): {
   "answer": 3
}
23
Response from model (Explaination): {
   "answer": 4
}
24
Response from model (Explaination): {
   "answer": 3
}
25
Response from model (Explaination): {
   "answer": 1
}
26
Response from model (Explaination): {
   "answer": 2
}
27
Response from model (Explaination): {
   "answer": 2
}
28
Response from model (Explaination): {
   "answer": 2
}
29


ERROR:root:Error during prediction: 'answer'


Response from model (Explaination): {
   "1": 3
}
30
Response from model (Explaination): {
   "answer": 3
}
31
Response from model (Explaination): {
   "answer": 1
}
32
Response from model (Explaination): {
   "answer": 1
}
33


ERROR:root:Error during prediction: 'answer'


Response from model (Explaination): {
   "1": 4
}
34
Response from model (Explaination): {
   "answer": 4
}
35


ERROR:root:Error during prediction: Error code: 400 - {'error': {'message': "Failed to generate JSON. Please adjust your prompt. See 'failed_generation' for more details.", 'type': 'invalid_request_error', 'code': 'json_validate_failed', 'failed_generation': '{\n   "1"\n}'}}


36
Response from model (Explaination): {
   "answer": 3
}
37
Response from model (Explaination): {
   "answer":3
}
38
Response from model (Explaination): {
   "answer": 1
}
39
Response from model (Explaination): {
   "answer": 3
}
40
Response from model (Explaination): {
   "answer": 3
}
41
Response from model (Explaination): {
   "answer": 4
}
42
Response from model (Explaination): {
   "answer": 2
}
43
Response from model (Explaination): {
   "answer": 3
}
44
Response from model (Explaination): {
   "answer": 1
}
45
Response from model (Explaination): {
   "answer": 3
}
46
Response from model (Explaination): {
   "answer": 2
}
47
Response from model (Explaination): {
   "answer": 1
}
48
Response from model (Explaination): {
   "answer": 3
}
49
Response from model (Explaination): {
   "answer": 4
}


,Passage,Question,Original Answer,Predicted Answer,Predicted Answer By Highlight,Predicted Answer By Own,Predicted Answer By Explaination,Option 1,Option 2,Option 3,Option 4,Explanation,Highlight,Keywords
0,بشر هزاران هزار سال از عدسی‌ها برای بزرگ‌نمایی...,کدام یک از موارد زیر را انسان‌ها در قرن شانزده...,4,4,NaN,4.0,4.0,استفاده از عدسی‌ها,بزرگنمایی اشیاء,ساخت عدسی‌ها,ساخت تلسکوپ,در متن به صراحت اشاره شده است که ساخت تلسکوپ ه...,با این حال، اولین تلسکوپ های واقعی در اواخر قر...,"['عدسی', 'بزرگنمایی', 'تلسکوپ', 'قرن شانزدهم',..."
1,گوارانی‌ها مهم‌ترین گروه بومی ساکن پاراگوئه شر...,دوران استعمارگری که سه قرن طول کشید توسط کدام ...,4,4,4.0,4.0,4.0,پاراگوئه,Guaycurú,گورانی‌ها,فاتحان اسپانیایی,در متن به صراحت ذکر شده است که دوره استعماری س...,اسپانیایی‌ها دوره استعماری را شروع کردند که سه...,"['فاتحان اسپانیایی', 'دوره استعماری', 'قرن 16'..."
2,خورشید پوسته‌ای شبیه کره زمین ندارد که بتوانید...,با توجه به متن، کدام قسمت از خورشید از زمین قا...,3,3,3.0,3.0,3.0,پوسته,پلاسما,فوتوسفر,مرکز,در متن به وضوح ذکر شده است که «وقتی به خورشید ...,وقتی به خورشید نگاه می‌کنیم قسمت بیرونی که می‌...,"['فوتوسفر', 'قسمت بیرونی', 'قابل مشاهده']"
3,آن مرد ظاهراً یک خودروی سه‌چرخ مسلح به مواد من...,مقامات چه اطلاعاتی درخصوص حمله تأیید کرده بودند؟,2,2,2.0,2.0,2.0,هویت هم‌دست‌ها,قومیت مظنون,انگیزه حمله,نام و نام خانوادگی مظنون,در متن ارائه شده، به دو مورد از اطلاعات تایید ...,مرد مظنون به انفجار بمب، پس از تحمل جراحات ناش...,"['مرد مظنون', 'بازداشت', 'گروه قومی اویغور']"
4,ما می توانیم زندگی دوستانه تری را با محیط زیست...,با توجه به متن، کدام مورد راه‌حلی قابل اجرا بر...,3,3,3.0,3.0,3.0,تحریم انتخاب‌های سبک زندگی مصرف‌گرایانه,کنشگری,یافتن ریشه مسئله,تبدیل شدن به متحد جنبش محیط زیست,در این متن، به دنبال راه‌حلی برای مسائل زیست‌م...,با این وجود، اگر تنها به دنبال یک راه‌حل موقت ...,"['راه\u200cحل موقت', 'ریشه مشکلات', 'فعالیت\u2..."
5,ورزش‌های زمستانی روی تپه، از جمله اسکی و اسنوب...,مطابق با متن، حدود چه زمانی اسکی‌بازی اولین با...,4,4,4.0,2.0,4.0,قرن هجدهم,5000 سال قبل از میلاد,1861,قرن هفدهم,در متن به صراحت اشاره شده است که ورزش اسکی روی...,ورزش اسکی روی تپه حداقل به قرن 17 ام برمی‌گردد,"['اسکی', 'ورزش', 'قرن 17 ام']"
6,ACMA همچنین نتیجه گرفت که علیرغم اینکه این وید...,برای اینکه رسانه تحت مقوله محتوای اینترنتی لحا...,2,2,2.0,2.0,2.0,باید از طریق اینترنت پخش شود,باید به صورت فیزیکی در سروری قرار گیرد,نمی‌تواند روی وبسایت ذخیره شود,باید توسط ACMA تأیید شود,در متن به صراحت بیان شده است که برای اینکه محت...,قانون خدمات پخش مقررات محتوای اینترنتی را مشخص...,"['قانون خدمات پخش', 'محتوای اینترنتی', 'سرور',..."
7,عرفان عبارت است از جستجوی ارتباط، شناخت یا درک...,با توجه به متن، کدام یک از موارد زیر مورد توجه...,2,2,2.0,2.0,2.0,جستجوی شهود به واقعیت الهی,طلب بینش از طریق تجربیات مشترک و همگانی,تأکید بر حالت سعادتمند خودآگاه,طلب درک آگاهانه الوهیت,در متن به صراحت بیان شده است که عرفان بر تجربه...,در عرفان بر تجربه مستقیم **شخصی** تأکید دارد.,"['تجربه مستقیم', 'حالت هشیاری', 'تجربه شخصی', ..."
8,محمد عمیقا به امور فراتر از این زندگی دنیوی عل...,بر اساس متن، غار محمد بیانگر چه چیزی درباره اع...,1,1,1.0,1.0,1.0,او برای آرامش و انزوا ارزش قائل بود,صدای بیرون به تأمل او کمک می‌کرد,به زمان خیلی کمی برای تنهایی نیاز داشت,نگران این بود که دیده یا شنیده شود,متن به وضوح بیان می‌کند که غار حرا، مکانی منزو...,در یک انزوا کامل خواهید بود,"['غار حرا', 'انزوا', 'آرامش', 'تأمل']"
9,کودکان از زمان کودکی نسبت به نژاد و کلیشه های ...,با توجه به نظریه ارائه شده در متن، برای کودکان...,2,2,2.0,2.0,2.0,کودک سعی می‌کند از درگیر شدن در رفتارهای تهدید...,احتمال بیشتری وجود دارد که کودک درگیر رفتارهای...,کودک در موقعیت‌های اجتماعی کناره‌گیری می‌کند,کودک از تعامل با سایر اقلیت‌های نژادی امتناع خ...,در این متن، نظریه‌ای مطرح می‌شود که نشان می‌ده...,کودکان از زمان کودکی نسبت به نژاد و کلیشه های ...,"['کلیشه نژادی', 'تأثیر بر رفتار', 'اقلیت نژادی..."


In [43]:
check_utility_df_2 = pd.read_csv('check_utility_2.csv')

In [47]:
import pandas as pd

# Assuming 'df' is the DataFrame containing your dataset

# Function to calculate accuracy
def calculate_accuracy(df):
    
    # Accuracy based on Original Answer and Predicted Answer By Highlight
    accuracy_original_vs_predicted_explaination = (df['Original Answer'] == df['Predicted Answer By Explaination']).mean()

    # Accuracy based on Original Answer and Predicted Answer By Highlight
    accuracy_original_vs_predicted_by_own = (df['Original Answer'] == df['Predicted Answer By Own']).mean()
    
    # Accuracy based on Original Answer and Predicted Answer By Own
    accuracy_predicted_vs_predicted_by_explaination = (df['Predicted Answer'] == df['Predicted Answer By Explaination']).mean()
    
    # Accuracy based on Original Answer and Predicted Answer By Own
    accuracy_predicted_vs_predicted_by_own = (df['Predicted Answer'] == df['Predicted Answer By Own']).mean()
    
    # Print or return the accuracies
    print(f"Accuracy (Original Answer vs Predicted Answer By Explaination): {accuracy_original_vs_predicted_explaination:.4f}")
    print(f"Accuracy (Original Answer vs Predicted Answer By Own): {accuracy_original_vs_predicted_by_own:.4f}")
    # Print or return the accuracies
    print(f"Accuracy (Predicted Answer vs Predicted Answer By Explaination): {accuracy_predicted_vs_predicted_by_explaination:.4f}")
    print(f"Accuracy (Predicted Answer vs Predicted Answer By Own): {accuracy_predicted_vs_predicted_by_own:.4f}")

    
    return accuracy_original_vs_predicted_explaination, accuracy_original_vs_predicted_by_own,accuracy_predicted_vs_predicted_by_explaination,accuracy_predicted_vs_predicted_by_own

# Example usage
_,_,_,_ = calculate_accuracy(check_utility_df_2)

Accuracy (Original Answer vs Predicted Answer By Explaination): 0.9000
Accuracy (Original Answer vs Predicted Answer By Own): 0.8000
Accuracy (Predicted Answer vs Predicted Answer By Explaination): 0.9000
Accuracy (Predicted Answer vs Predicted Answer By Own): 0.8200
